# Select and Ultimate

In [40]:
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Union
from dataclasses import dataclass
from datetime import datetime

@dataclass
class ActuarialParams:
    """Parameters for actuarial calculations"""
    i: float  # Interest rate
    m: int    # Payment frequency
    sel: float  # Selection factor
    A: float   # Mortality parameter A
    B: float   # Mortality parameter B
    c: float   # Mortality parameter c

    def __post_init__(self):
        """Validate parameters after initialization"""
        if not 0 <= self.i <= 1:
            raise ValueError("Interest rate must be between 0 and 1")
        if self.m <= 0:
            raise ValueError("Payment frequency must be positive")
        if not 0 <= self.sel <= 1:
            raise ValueError("Selection factor must be between 0 and 1")
        if self.A < 0 or self.B < 0:
            raise ValueError("Mortality parameters A and B must be non-negative")
        if self.c <= 1:
            raise ValueError("Mortality parameter c must be greater than 1")

class ActuarialCalculator:
    """
    Advanced actuarial calculator for life insurance and annuity calculations.
    
    Implements various actuarial calculations including:
    - Survival probabilities
    - Life expectancies
    - Annuity values (immediate and due)
    - Insurance values
    - Deferred benefits
    """
    
    def __init__(self, params: Optional[ActuarialParams] = None):
        """
        Initialize calculator with given parameters or defaults.
        
        Args:
            params: ActuarialParams instance containing calculation parameters
        """
        if params is None:
            params = ActuarialParams(
                i=0.05, m=12, sel=0.9, 
                A=0.00022, B=0.0000027, c=1.124
            )
        self.params = params
        
        # Derived parameters
        self.v = 1 / (1 + params.i)
        self.d_m = params.m * (1 - self.v**(1/params.m))
        self.delta = np.log(1 + params.i)
        
        # Cache for calculations
        self._cache = {}

    def _calculate_survival_probability(self, x: int) -> float:
        """
        Calculate survival probability for age x.
        
        Args:
            x: Age
            
        Returns:
            Survival probability
        """
        if x < 20:
            return np.exp(self.params.sel * (
                self.params.A/np.log(self.params.sel)*(1-self.params.sel) - 
                self.params.B*self.params.c**x*(self.params.c-self.params.sel)/np.log(self.params.c/self.params.sel)
            ))
        return np.exp(-(
            self.params.A + self.params.B/np.log(self.params.c)*self.params.c**x*(self.params.c-1)
        ))

    def calculate_all(self, start_age: int = 18, end_age: int = 28) -> pd.DataFrame:
        """
        Calculate all actuarial values for given age range.
        
        Args:
            start_age: Starting age for calculations
            end_age: Ending age for calculations
            
        Returns:
            DataFrame containing all calculated values
            
        Raises:
            ValueError: If start_age or end_age are invalid
        """
        if not 0 <= start_age <= end_age:
            raise ValueError("Invalid age range")
        if end_age > 150:  # Reasonable maximum age
            raise ValueError("End age too high")
            
        # Initialize calculation ranges
        ages = list(range(start_age, end_age + 1))
        max_age = end_age + 1
        
        # Calculate and cache basic probabilities
        px = {x: self._calculate_survival_probability(x) 
              for x in range(start_age - 1, max_age + 1)}
        px_prev = {x: self._calculate_survival_probability(x) 
                  for x in range(start_age - 1, max_age + 1)}
        
        # Calculate lives (l_x) with more precise initial value
        lx = {20: 100000}  # Standard initial value
        for x in range(19, start_age - 1, -1):
            lx[x] = lx[x + 1] / px[x]
        for x in range(20, max_age):
            lx[x + 1] = lx[x] * px[x]
        
        # Calculate annuity values
        ax = self._calculate_annuities(px, max_age, start_age)
        ax_prev = self._calculate_annuities(px_prev, max_age, start_age)
        ax_m = self._calculate_monthly_annuities(ax, px, max_age, start_age)
        ax_m_prev = self._calculate_monthly_annuities(ax_prev, px_prev, max_age, start_age)
        
        # Calculate insurance values
        Ax = {x: 1 - (1 - self.v) * ax[x] 
              for x in range(start_age, max_age)}
        Ax_prev = {x: 1 - (1 - self.v) * ax_prev[x] 
                  for x in range(start_age, max_age)}
        Am = {x: 1 - self.d_m * ax_m[x] 
              for x in range(start_age, max_age)}
        
        # Calculate deferred benefits and term annuities
        Ex = self._calculate_deferred_benefits(ages, max_age, lx)
        ax_term = self._calculate_term_annuities(ages, ax, Ex)
        
        # Create results DataFrame with metadata
        data = self._create_results_dataframe(
            ages, px, px_prev, lx, ax, ax_prev, ax_m, ax_m_prev,
            Ax, Ax_prev, Am, Ex, ax_term
        )
        
        return data

    def _calculate_annuities(self, px: Dict[int, float], max_age: int, 
                           start_age: int) -> Dict[int, float]:
        """Calculate standard annuity values"""
        ax = {max_age: 1}
        for x in range(max_age - 1, start_age - 1, -1):
            ax[x] = 1 + self.v * px[x] * ax[x + 1]
        return ax

    def _calculate_monthly_annuities(self, ax: Dict[int, float], 
                                   px: Dict[int, float], max_age: int, 
                                   start_age: int) -> Dict[int, float]:
        """Calculate monthly annuity values with corrections"""
        ax_m = {max_age: 1}
        for x in range(max_age - 1, start_age - 1, -1):
            correction = ((self.params.m - 1)/(2*self.params.m) + 
                         (self.params.m**2 - 1)/(12*self.params.m**2) * 
                         (self.delta + self.params.sel**2 * 
                          (self.params.A + self.params.B*self.params.c**x)))
            ax_m[x] = ax[x] - correction
        return ax_m

    def _calculate_deferred_benefits(self, ages: List[int], max_age: int, 
                                   lx: Dict[int, float]) -> Dict[int, Dict[int, float]]:
        """Calculate deferred benefits for various periods"""
        Ex = {}
        for x in ages:
            Ex[x] = {}
            for n in [10, 20, 25]:
                if x + n <= max_age - 1:
                    Ex[x][n] = self.v**n * lx[x + n] / lx[x]
        return Ex

    def _calculate_term_annuities(self, ages: List[int], ax: Dict[int, float], 
                                Ex: Dict[int, Dict[int, float]]) -> Dict[int, Dict[int, float]]:
        """Calculate term annuities for various periods"""
        ax_term = {}
        for x in ages:
            ax_term[x] = {}
            for n in [10, 20]:
                if n in Ex[x]:
                    ax_term[x][n] = ax[x] - Ex[x][n] * ax[x + n]
        return ax_term

    def _create_results_dataframe(self, ages: List[int], px: Dict[int, float], 
                                px_prev: Dict[int, float], lx: Dict[int, float], 
                                ax: Dict[int, float], ax_prev: Dict[int, float], 
                                ax_m: Dict[int, float], ax_m_prev: Dict[int, float],
                                Ax: Dict[int, float], Ax_prev: Dict[int, float], 
                                Am: Dict[int, float], Ex: Dict[int, Dict[int, float]], 
                                ax_term: Dict[int, Dict[int, float]]) -> pd.DataFrame:
        """Create and format results DataFrame"""
        data = []
        for x in ages:
            row = {
                'age': x,
                'p[x]': px[x],
                'p[x-1]+1': px_prev[x],
                'px': px[x] if x >= 20 else None,
                'l[x]': lx[x],
                'ä[x]': ax[x],
                'ä[x-1]+1': ax_prev[x],
                'äx': ax[x] if x >= 20 else None,
                'ä(m)[x]': ax_m[x],
                'ä(m)[x-1]+1': ax_m_prev[x],
                'ä(m)x': ax_m[x] if x >= 20 else None,
                'A[x]': Ax[x],
                'A[x-1]+1': Ax_prev[x],
                'Ax': Ax[x] if x >= 20 else None,
                'A(m)x': Am[x],
                '10E[x]': Ex[x].get(10),
                '20E[x]': Ex[x].get(20),
                '25E[x]': Ex[x].get(25),
                'ä[x]:10-years': ax_term[x].get(10),
                'ä[x]:20-years': ax_term[x].get(20)
            }
            data.append(row)
            
        df = pd.DataFrame(data)
        df.insert(0, 'calculation_date', datetime.now().date())
        return df

def display_results(df: pd.DataFrame, precision: int = 6) -> None:
    """
    Display results with proper formatting
    
    Args:
        df: DataFrame containing actuarial calculations
        precision: Number of decimal places to display
    """
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.precision', precision)
    pd.set_option('display.float_format', 
                 lambda x: f'{x:.{precision}f}' if isinstance(x, float) else str(x))
    
    # Format integer columns
    integer_cols = ['age', 'l[x]']
    for col in integer_cols:
        if col in df.columns:
            df[col] = df[col].astype(int)
    
    print(df)

def main():
    """Main function to run actuarial calculations"""
    try:
        # Initialize calculator with default parameters
        calc = ActuarialCalculator()
        
        # Compute results
        results = calc.calculate_all(start_age=18, end_age=130)
        
        # Display results
        display_results(results)
        
    except Exception as e:
        print(f"Error occurred during calculation: {str(e)}")

# First run main() and capture the results
calc = ActuarialCalculator()
results = calc.calculate_all(start_age=18, end_age=130)

# Then display them
display_results(results)        


    calculation_date  age     p[x]  p[x-1]+1       px    l[x]      ä[x]  \
0         2024-11-26   18 0.999792  0.999792      NaN  100041 20.054713   
1         2024-11-26   19 0.999790  0.999790      NaN  100021 20.011611   
2         2024-11-26   20 0.999750  0.999750 0.999750  100000 19.966394   
3         2024-11-26   21 0.999747  0.999747 0.999747   99975 19.919686   
4         2024-11-26   22 0.999743  0.999743 0.999743   99949 19.870704   
..               ...  ...      ...       ...      ...     ...       ...   
108       2024-11-26  126 0.000795  0.000795 0.000795       0  1.000757   
109       2024-11-26  127 0.000328  0.000328 0.000328       0  1.000312   
110       2024-11-26  128 0.000121  0.000121 0.000121       0  1.000115   
111       2024-11-26  129 0.000040  0.000040 0.000040       0  1.000038   
112       2024-11-26  130 0.000011  0.000011 0.000011       0  1.000011   

     ä[x-1]+1        äx   ä(m)[x]  ä(m)[x-1]+1     ä(m)x     A[x]  A[x-1]+1  \
0   20.054713       

In [44]:

age_mortality= results[results['age'] == 31]
age_mortality

,calculation_date,age,p[x],p[x-1]+1,px,l[x],ä[x],ä[x-1]+1,äx,ä(m)[x],ä(m)[x-1]+1,ä(m)x,A[x],A[x-1]+1,Ax,A(m)x,10E[x],20E[x],25E[x],ä[x]:10-years,ä[x]:20-years
13,2024-11-26,31,0.999673,0.999673,0.999673,99695,19.308620,19.308620,19.308620,18.846227,18.846227,18.846227,0.080542,0.080542,0.080542,0.082356,0.611389,0.372207,0.289247,8.095556,13.038374


# Ultimate

In [33]:
import numpy as np
import pandas as pd
from dataclasses import dataclass
from typing import Dict, List, Optional

@dataclass
class SUSMParams:
    """Parameters for SUSM calculations"""
    i: float = 0.05    # Interest rate
    m: int = 12        # Payment frequency
    A: float = 0.00022 # Mortality parameter A
    B: float = 0.0000027 # Mortality parameter B
    cc: float = 1.124  # Mortality parameter c (renamed to cc to avoid confusion with math.c)

class SUSMCalculator:
    """
    SUSM (Select and Ultimate Survival Model) Calculator for actuarial calculations.
    Implements annuity and insurance function calculations.
    """
    
    def __init__(self, params: Optional[SUSMParams] = None):
        """Initialize calculator with parameters"""
        self.params = params or SUSMParams()
        
        # Derived parameters
        self.v = 1 / (1 + self.params.i)
        self.d_m = self.params.m * (1 - self.v**(1/self.params.m))
        self.delta = np.log(1 + self.params.i)

    def calculate_px(self, x: int) -> float:
        """Calculate survival probability px"""
        return np.exp(-(
            self.params.A + 
            self.params.B/np.log(self.params.cc) * 
            self.params.cc**x * (self.params.cc-1)
        ))

    def calculate_lx(self, x: int, initial_lives: int = 100000) -> float:
        """Calculate number of lives at age x"""
        if x == 20:
            return initial_lives
        
        lx = initial_lives
        if x > 20:
            for t in range(20, x):
                lx *= self.calculate_px(t)
        else:
            for t in range(x, 20):
                lx /= self.calculate_px(t)
        return lx

    def calculate_ax(self, x: int) -> float:
        """Calculate whole life annuity-due value"""
        max_age = 130  # Terminal age
        ax = 1.0
        v_power = self.v
        
        for t in range(x + 1, max_age):
            survival_prob = 1.0
            for k in range(x, t):
                survival_prob *= self.calculate_px(k)
            ax += v_power * survival_prob
            v_power *= self.v
            
        return ax

    def calculate_ax_term(self, x: int, n: int) -> float:
        """Calculate n-year term annuity-due value"""
        ax = 1.0
        v_power = self.v
        
        for t in range(x + 1, x + n):
            survival_prob = 1.0
            for k in range(x, t):
                survival_prob *= self.calculate_px(k)
            ax += v_power * survival_prob
            v_power *= self.v
            
        return ax

    def calculate_ax_m(self, x: int) -> float:
        """Calculate monthly annuity-due value"""
        ax = self.calculate_ax(x)
        correction = (
            (self.params.m - 1)/(2*self.params.m) + 
            (self.params.m**2 - 1)/(12*self.params.m**2) * 
            (self.delta + self.params.A + self.params.B*self.params.cc**x)
        )
        return ax - correction

    def calculate_Ax(self, x: int) -> float:
        """Calculate whole life insurance value"""
        ax = self.calculate_ax(x)
        return 1 - (1 - self.v) * ax
    
    def calculate_Ax_term(self, x: int, n: int) -> float:
        """Calculate n-year term insurance value"""
        ax_n = self.calculate_ax_term(x, n)
        nEx = self.calculate_nEx(x, n)
        return 1 - (1 - self.v) * ax_n - self.v**n * nEx

    def calculate_Am(self, x: int) -> float:
        """Calculate monthly insurance value"""
        ax_m = self.calculate_ax_m(x)
        return 1 - self.d_m * ax_m

    def calculate_nEx(self, x: int, n: int) -> float:
        """Calculate n-year pure endowment value"""
        return (self.v**n * 
                self.calculate_lx(x + n) / 
                self.calculate_lx(x))

    def calculate_2ax(self, x: int) -> float:
        """
        Calculate the present value of a whole life annuity-due (2ax)
        using double the force of interest (v^2).
        """
        max_age = 130  # Terminal age
        ax = 1.0
        v_power = self.v**2  # Use v^2 for double the force of interest
        
        for t in range(x + 1, max_age):
            survival_prob = 1.0
            for k in range(x, t):
                survival_prob *= self.calculate_px(k)
            ax += v_power * survival_prob
            v_power *= self.v**2  # Keep updating with v^2
        
        return ax


    def calculate_2Ax(self, x: int) -> float:
        """Calculate 2-year insurance value"""
        ax_2 = self.calculate_2ax(x)
        return 1 - (1 - self.v**2) * ax_2

    def calculate_all(self, x: int) -> Dict[str, float]:
        """Calculate all values for a given age"""
        results = {
            'age': x,
            'px': self.calculate_px(x),
            'lx': self.calculate_lx(x),
            'ax': self.calculate_ax(x),
            'ax_10': self.calculate_ax_term(x, 10),
            'Ax_10': self.calculate_Ax_term(x, 10),
            'ax_20': self.calculate_ax_term(x, 20),
            'Ax_20': self.calculate_Ax_term(x, 20),
            'ax_m': self.calculate_ax_m(x),
            'Ax': self.calculate_Ax(x),
            'Am': self.calculate_Am(x),
            '10Ex': self.calculate_nEx(x, 10),
            '20Ex': self.calculate_nEx(x, 20),
            '2ax': self.calculate_2ax(x),
            '2Ax': self.calculate_2Ax(x),
            '5Ex': self.calculate_nEx(x, 5),
            '10Ex': self.calculate_nEx(x, 10),
            '20Ex': self.calculate_nEx(x, 20)
        }
        return results

def generate_table(start_age: int = 20, end_age: int = 130) -> pd.DataFrame:
    """Generate a table of all values for a range of ages"""
    calc = SUSMCalculator()
    data = []
    
    for x in range(start_age, end_age + 1):
        results = calc.calculate_all(x)
        data.append(results)
    
    df = pd.DataFrame(data)
    return df

def display_table(df: pd.DataFrame, precision: int = 6) -> None:
    """Display results table with formatting"""
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.precision', precision)
    pd.set_option('display.float_format', 
                 lambda x: f'{x:.{precision}f}' if isinstance(x, float) else str(x))
    
    # Format integer columns
    integer_cols = ['age', 'lx']
    for col in integer_cols:
        if col in df.columns:
            df[col] = df[col].astype(int)
    
    print(df)

def main():
    """Main function to generate and display results"""
    try:
        # Generate table for ages 20-130
        results = generate_table(20, 130)
        
        # Display results
        print("\nSUSM Calculator Results:")
        print("=======================")
        display_table(results, precision=4)
        
        # Save to CSV
        results.to_csv('susm_results.csv')
        print("\nResults have been saved to 'susm_results.csv'")
        
    except Exception as e:
        print(f"Error occurred during calculation: {str(e)}")

if __name__ == "__main__":
    main()


SUSM Calculator Results:
     age     px      lx      ax  ax_10  Ax_10   ax_20  Ax_20    ax_m     Ax  \
0     20 0.9998  100000 19.9664 8.0991 0.2385 13.0559 0.2372 19.5040 0.0492   
1     21 0.9997   99975 19.9197 8.0990 0.2385 13.0551 0.2373 19.4573 0.0514   
2     22 0.9997   99949 19.8707 8.0988 0.2385 13.0541 0.2374 19.4083 0.0538   
3     23 0.9997   99923 19.8193 8.0986 0.2386 13.0531 0.2375 19.3569 0.0562   
4     24 0.9997   99897 19.7655 8.0983 0.2386 13.0519 0.2376 19.3031 0.0588   
..   ...    ...     ...     ...    ...    ...     ...    ...     ...    ...   
106  126 0.0008       0  1.0008 1.0008 0.9523  1.0008 0.9523 -0.0184 0.9523   
107  127 0.0003       0  1.0003 1.0003 0.9524  1.0003 0.9524 -0.0879 0.9524   
108  128 0.0001       0  1.0001 1.0001 0.9524  1.0001 0.9524 -0.1657 0.9524   
109  129 0.0000       0  1.0000 1.0000 0.9524  1.0000 0.9524 -0.2531 0.9524   
110  130 0.0000       0  1.0000 1.0000 0.9524  1.0000 0.9524 -0.3511 0.9524   

        Am   10Ex   20Ex 

# Interest Functions

In [37]:
import numpy as np
import pandas as pd

def calculate_table(i=0.05):
    """Calculate complete interest rate table for given nominal rate i"""
    m_values = [1, 2, 4, 12, float('inf')]
    results = []
    
    for m in m_values:
        # Calculate i(m)
        if np.isinf(m):
            i_m = np.log(1 + i)
            d_m = i_m  # For continuous compounding, d(∞) = i(∞)
        else:
            i_m = m * ((1 + i)**(1/m) - 1)
            d_m = i_m / (1 + i_m/m)
        
        # Calculate effective rates
        i_ratio = i / i_m
        
        # Calculate effective discount rate
        d = i / (1 + i)
        d_ratio = d / d_m
        
        # Calculate α(m)
        alpha = (i * d_m) / (i_m * d)
        
        # Calculate β(m)
        beta = (i - i_m) / (i_m * d_m)
        
        results.append({
            'm': m,
            'i(m)': round(i_m, 5),
            'd(m)': round(d_m, 5),
            'i/i(m)': round(i_ratio, 5),
            'd/d(m)': round(d_ratio, 5),
            'α(m)': round(alpha, 5),
            'β(m)': round(beta, 5)
        })
    
    return pd.DataFrame(results)

# Calculate and display the table
table = calculate_table(0.05)
pd.set_option('display.float_format', lambda x: '{:.5f}'.format(x))
print("\nCalculated Interest Rate Table:")
print(table.to_string(index=False))



Calculated Interest Rate Table:
       m    i(m)    d(m)  i/i(m)  d/d(m)    α(m)     β(m)
 1.00000 0.05000 0.04762 1.00000 1.00000 1.00000 -0.00000
 2.00000 0.04939 0.04820 1.01235 0.98795 1.02470  0.25617
 4.00000 0.04909 0.04849 1.01856 0.98196 1.03727  0.38272
12.00000 0.04889 0.04869 1.02271 0.97798 1.04574  0.46651
     inf 0.04879 0.04879 1.02480 0.97600 1.05000  0.50823


In [38]:
import numpy as np
from scipy.stats import norm

def generate_z_table():
    # Create row values (0.0 to 3.9)
    row_values = np.arange(0, 4.0, 0.1)
    # Create column values (0.00 to 0.09)
    col_values = np.arange(0, 0.10, 0.01)
    
    print("*z*", end="")
    # Print column headers
    for col in col_values:
        print(f"{col:.2f}", end="")
    print()
    
    # Generate each row
    for row in row_values:
        print(f"{row:.1f}", end="")
        for col in col_values:
            z_value = row + col
            # Calculate probability using cumulative distribution function
            probability = norm.cdf(z_value)
            print(f"{probability:.4f}", end="")
        print()

# Generate the table
generate_z_table()

*z*0.000.010.020.030.040.050.060.070.080.09
0.00.50000.50400.50800.51200.51600.51990.52390.52790.53190.5359
0.10.53980.54380.54780.55170.55570.55960.56360.56750.57140.5753
0.20.57930.58320.58710.59100.59480.59870.60260.60640.61030.6141
0.30.61790.62170.62550.62930.63310.63680.64060.64430.64800.6517
0.40.65540.65910.66280.66640.67000.67360.67720.68080.68440.6879
0.50.69150.69500.69850.70190.70540.70880.71230.71570.71900.7224
0.60.72570.72910.73240.73570.73890.74220.74540.74860.75170.7549
0.70.75800.76110.76420.76730.77040.77340.77640.77940.78230.7852
0.80.78810.79100.79390.79670.79950.80230.80510.80780.81060.8133
0.90.81590.81860.82120.82380.82640.82890.83150.83400.83650.8389
1.00.84130.84380.84610.84850.85080.85310.85540.85770.85990.8621
1.10.86430.86650.86860.87080.87290.87490.87700.87900.88100.8830
1.20.88490.88690.88880.89070.89250.89440.89620.89800.89970.9015
1.30.90320.90490.90660.90820.90990.91150.91310.91470.91620.9177
1.40.91920.92070.92220.92360.92510.92650.92790.92920.93060.9